### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [1]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [2]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位经验丰富的生物学家。你特别擅长解答有关生物学的问题。
你之所以能够出色地回答问题，是因为你对生物学的各个分支都有深入的了解，
包括细胞生物学、遗传学、生态学和进化论。

这是一个问题：
{input}"""

computer_template = """你是一位杰出的计算机科学家。你在处理计算机科学相关问题方面非常专业。
你能够出色完成任务的秘密在于你对算法、数据结构、编程语言和系统架构的深刻理解。

这是一个问题：
{input}"""

chinese_template = """你是一位才华横溢的汉语言文学老师。你对古典文学和现代文学都有着深厚的理解。
你能够精彩地回答有关文学的问题，这得益于你对文学作品的深入分析，
以及你对诗歌、散文、小说等各种文学形式的广泛知识。

这是一个问题：
{input}"""

In [3]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物学问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_template,
    },

]

In [4]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [5]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [24]:
print(destination_chains)

{'物理': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='你是一位非常聪明的物理教授。\n你擅长以简洁易懂的方式回答关于物理的问题。\n当你不知道某个问题的答案时，你会坦诚承认。\n\n这是一个问题：\n{input}'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x000001C17FF132D0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x000001C10E025910>, openai_api_key=SecretStr('**********'), openai_proxy='')), '数学': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='你是一位很棒的数学家。你擅长回答数学问题。\n之所以如此出色，是因为你能够将难题分解成各个组成部分，\n先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。\n\n这是一个问题：\n{input}'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x000001C17FF132D0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x000001C10E025910>, openai_api_key=SecretStr('**********'), openai_proxy='')), '生物': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='你是一位经验丰富的生物学家。你特别擅长解答有关生物学的问题。\n你之所以能够出色地回答问题，是因为你对生物学的各个分支都有深入的了解，\n包括细胞生物学、遗传学、生态学和进化论。\n\n这是一个问题：\n{inp

In [6]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [7]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [8]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [9]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物学问题', '计算机: 适用于回答计算机问题', '汉语言文学: 适用于回答汉语言文学问题']


In [10]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题


In [25]:
print(router_chain)

verbose=True llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['input'], output_parser=RouterOutputParser(), template='Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the original input if you don\'t think any modificati

In [11]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [12]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}



In [13]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [14]:
print(chain.run("GPU是什么?"))

d:\anacondaInstall\envs\transformers\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new MultiPromptChain chain...
计算机: {'input': 'GPU是什么?'}
> Finished chain.


GPU是图形处理器，也称为显卡，是一种专门用于处理图形和图像相关计算的硬件设备。它可以加速计算机图形渲染、视频编码、深度学习等任务，相较于传统的中央处理器（CPU），GPU具有更强大的计算能力和并行处理能力。它通常被使用在游戏、设计、科学计算等领域。


In [15]:
print(
    chain.run(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.


答案：43。因为43是大于40的最小质数，43+1=44，44能被3整除。


In [16]:
print(chain.run("矛盾文学奖是哪一年设置的?"))



> Entering new MultiPromptChain chain...
汉语言文学: {'input': '矛盾文学奖是哪一年设置的?'}
> Finished chain.


矛盾文学奖是1981年在中国由中国作家协会设立的文学奖项。它旨在表彰在社会主义现实主义文学创作中，具有独立思考、反映矛盾和冲突的作品。自1981年至今，矛盾文学奖已经评选出多位优秀作家，为中国文学界做出了重要贡献。


In [22]:
print(chain.run("诗书礼易乐春秋是指什么?"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言文学: {'input': '诗书礼易乐春秋是指什么?'}
> Finished chain.


诗书礼易乐春秋是中国古代一部重要的典籍，也是儒家经典之一。它是由儒家思想家孔子及其弟子编纂的一部政治、道德和文化方面的著作，内容涉及诗、书、礼、易、乐等方面的知识。它强调的是君子应该具备的品德和修养，以及如何治理国家和社会。它被认为是传统文化的重要组成部分，对中国古代的政治、社会和文化产生了深远的影响。


In [17]:
print(chain.run("大象的鼻子有什么作用"))



> Entering new MultiPromptChain chain...
生物: {'input': '大象的鼻子有什么作用'}
> Finished chain.
？

大象的鼻子是一种非常重要的器官，它在大象的生活中发挥着多种作用。首先，大象的鼻子是它们的主要呼吸器官，它们可以通过鼻子吸入氧气并排出二氧化碳。此外，大象的鼻子也是它们的嗅觉器官，它们可以借助鼻子来感知周围的气味，包括食物、水源、同类和潜在的危险。大象的鼻子还有助于它们在炎热的天气中保持体温平衡，因为它们可以通过鼻子排出热量。最后，大象的鼻子还能用来触摸和抓取物体，帮助它们在采食和社交


In [18]:
router_chain.verbose = True

In [19]:
print(chain.run("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.


黑洞是一种极为密集的天体，它的引力极强，甚至连光都无法逃脱。它的形成是由于某颗恒星燃尽所有的燃料，塌缩成一个无限小、无限密集的点，这就是我们所说的黑洞。它的引力场非常强大，可以吸引周围的物质，甚至连光也无法逃脱，因此我们无法直接观测到它。但是，科学家通过观测周围的物质运动和引力效应，可以推断出黑洞的存在。黑洞是宇宙中非常神秘和有趣的物体，它们的研究也是物理学家们持续探索的重要课题。
